In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType() , True),   
                                 StructField("surname", StringType(), True)])

In [0]:
driver_schema = StructType([StructField("driverId", IntegerType(), False), 
                            StructField("driverRef",StringType(), False),
                            StructField("number", IntegerType(), False),
                            StructField("code", StringType(), False),
                            StructField("name", name_schema, False),
                            StructField("dob", DateType(), False),
                            StructField("nationality", StringType(), False),
                            StructField("url", StringType(), False)])

In [0]:
drivers_df = spark.read \
    .schema(driver_schema)  \
    .json("/mnt/dldatabrickscoursedev001/raw/drivers.json")

In [0]:
drivers_df.printSchema()

In [0]:
drivers_dropped_df = drivers_df.drop("url")

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit, concat

In [0]:
drivers_final_df = drivers_dropped_df.withColumnRenamed("driverId","driver_id")   \
                                              .withColumnRenamed("driverRef","driver_ref")  \
                                              .withColumn("ingestion_date",current_timestamp())  \
                                              .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/dldatabrickscoursedev001/processed/drivers/")

In [0]:
spark.read.parquet("/mnt/dldatabrickscoursedev001/processed/drivers/").display()